In [2]:
setwd('/Users/chansoosong/Desktop/Research/edsp2019project-chansooligans/Progress_Reports/')
library(rjson)
library(stringr)

# Measuring Judge Ideology

### Background:
Example: Martin-Quinn Scores, it's computed using a dynamic item-response model. It's *dynamic* because it allows for the latent ideology measure to vary across time. They take a Bayesian approach and achieve the *dynamic* part using a hierarchical random walk prior on the latent ideology measure. 

![Martin_Quinn](https://mqscores.lsa.umich.edu/images/ipAnim1937_2006.gif)

![Image1](Progress_Reports/images/img1.png)

## Using text to do the same

Consider Snyder vs. Phelps (2011):
- Quick facts: Westboro church members picketed with egregious homophobic signs at the funeral service of Marine Lance Corporal Matthew Snyder
- Briefly, Roberts states there are "special protections" for first amendment for speech regarding public issues that takes place on public land.

![Roberts](Progress_Reports/images/roberts.png)

- Alito dissents that free speech is not a license to inflict harm. Here's a passage:

![Alito](Progress_Reports/images/alito.png)

## Modeling Challenges:

**Legal Language**: Legal language is generally jargony and typically avoids colloquialisms (e.g. in Congress, members use colloquial speech in floor speeches often such as "death tax", "fake news", "alternative facts", "pro-life", "anti-choice").

Opinion Structure:
1. Fact Pattern + How Case Got to This Court
2. Explain Legal Concepts, Explain Precedents
3. Apply Legal Analysis
4. Summarize


## Data Challenges:

Example #1: https://www.courtlistener.com/opinion/2075/greenwich-financial-services-distressed-mortgage-f/

In [3]:
example1 = fromJSON(file='data/2448.json')
lapply(example1, function(x) substr(x,0,1000))

$resource_uri
[1] "http://www.courtlistener.com/api/rest/v3/opinions/2448/"

$absolute_url
[1] "/opinion/2448/wilson-v-cia/"

$cluster
[1] "http://www.courtlistener.com/api/rest/v3/clusters/2448/"

$author
character(0)

$joined_by
character(0)

$author_str
[1] ""

$per_curiam
[1] "FALSE"

$date_created
[1] "2010-04-24T18:46:13Z"

$date_modified
[1] "2017-03-24T00:07:39.383027Z"

$type
[1] "010combined"

$sha1
[1] "3e1df8de37ca3f53dd845a6600ee5757082b81d6"

$page_count
[1] "55"

$download_url
[1] "http://www.ca2.uscourts.gov/decisions/isysquery/9c4d23a6-1c14-4a46-97ee-8c7a979737b8/43/doc/07-4244-cv_opn.pdf"

$local_path
[1] "pdf/2009/11/12/Wilson_v._CIA.pdf"

$plain_text
[1] "07-4244-cv\nWilson v. CIA\n\n                             UNITED STATES COURT OF APPEALS\n\n                                     F OR THE S ECOND C IRCUIT\n\n\n                                        August Term, 2008\n\n         (Argued: January 8, 2009                         Decided: November 12, 2009)\n\n                                      Docket No. 07-4244-cv\n\n\n                         V ALERIE P LAME W ILSON, S IMON & S CHUSTER INC.,\n\n                                                                         Plaintiffs-Appellants,\n                                              —v.—\n\n  T HE C ENTRAL INTELLIGENCE A GENCY; L EON E. P ANETTA, in his official capacity as\nDirector of the Central Intelligence Agency; and D ENNIS C. B LAIR, in his official capacity\n                           as Director of National Intelligence,*\n\n                                                                       Defendants-Appellees.\n\n\nBefore:\n\n                K ATZMANN and R AGGI, Circuit Judges, and K EENAN, "

$html
[1] ""

$html_lawbox
[1] "<div>\n<center><b>586 F.3d 171 (2009)</b></center>\n<center><h1>Valerie Plame WILSON, Simon &amp; Schuster Inc., Plaintiffs-Appellants,<br>\nv.<br>\nThe CENTRAL INTELLIGENCE AGENCY; Leon E. Panetta, in his official capacity as Director of the Central Intelligence Agency; and Dennis C. Blair, in his official capacity as Director of National Intelligence,<sup>[*]</sup><span class=\"star-pagination\">*172</span> Defendants-Appellees.</h1></center>\n<center>Docket No. 07-4244-cv.</center>\n<center><p><b>United States Court of Appeals, Second Circuit.</b></p></center>\n<center>Argued: January 8, 2009.</center>\n<center>Decided: November 12, 2009.</center>\n<p><span class=\"star-pagination\">*173</span> David B. Smallman, Smallman &amp; Hans LLP, New York, NY, for Plaintiffs-Appellants.</p>\n<p>Benjamin H. Torrance, Assistant United States Attorney (Beth E. Goldman, Assistant United States Attorney, on the brief), for Michael J. Garcia, United States Attorney for the Southern District of New York, New Yor"

$html_columbia
character(0)

$html_with_citations
[1] "<div>\n<center><b><span class=\"citation no-link\"><span class=\"volume\">586</span> <span class=\"reporter\">F.3d</span> <span class=\"page\">171</span></span> (2009)</b></center>\n<center><h1>Valerie Plame WILSON, Simon &amp; Schuster Inc., Plaintiffs-Appellants,<br>\nv.<br>\nThe CENTRAL INTELLIGENCE AGENCY; Leon E. Panetta, in his official capacity as Director of the Central Intelligence Agency; and Dennis C. Blair, in his official capacity as Director of National Intelligence,<sup>[*]</sup><span class=\"star-pagination\">*172</span> Defendants-Appellees.</h1></center>\n<center>Docket No. 07-4244-cv.</center>\n<center><p><b>United States Court of Appeals, Second Circuit.</b></p></center>\n<center>Argued: January 8, 2009.</center>\n<center>Decided: November 12, 2009.</center>\n<p><span class=\"star-pagination\">*173</span> David B. Smallman, Smallman &amp; Hans LLP, New York, NY, for Plaintiffs-Appellants.</p>\n<p>Benjamin H. Torrance, Assistant United States Attorney (Beth E. Goldman, Assistant United St"

$extracted_by_ocr
[1] "FALSE"

$opinions_cited
 [1] "http://www.courtlistener.com/api/rest/v3/opinions/105087/" 
 [2] "http://www.courtlistener.com/api/res

### Problem 1: "Year" is missing  

Easy to get from "local path" if available, otherwise need to get from opinion.

In [4]:
example1$local_path

[1] "pdf/2009/11/12/Wilson_v._CIA.pdf"

In [5]:
file_names = list.files('data/')
opinions = list()
for(i in 1:length(file_names)) opinions[[i]] = fromJSON(file=paste('data/',file_names[i],sep=''))
lapply(opinions,function(x) x$local_path)

[[1]]
[1] "pdf/2013/08/01/manning_v._boston_medical_center.pdf"

[[2]]
[1] "pdf/2010/04/20/Greenwich_Financial_Services_Distressed_Mortgage_Fund_3_LLC_v._Country_.pdf"

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

[[12]]
NULL

[[13]]
NULL

[[14]]
[1] "pdf/2010/01/28/Keach_v._County_of_Schenectady.pdf"

[[15]]
[1] "pdf/2009/11/12/Wilson_v._CIA.pdf"

[[16]]
[1] "pdf/2009/10/15/Seidemann_v._Bowen.pdf"

[[17]]
[1] "pdf/2009/09/21/Lindsay_v._Assrsquon_of_Profrsquol_Flight_Attendants.pdf"

[[18]]
[1] "pdf/2009/03/26/a_DSMP_Holdings_.pdf"

[[19]]
[1] "pdf/2009/01/13/Zheng_v._Mukasey.pdf"

[[20]]
[1] "pdf/2008/10/16/USA_v._Luna_USA.pdf"

[[21]]
[1] "pdf/2008/09/24/In_Re_James__Karen_Wornick.pdf"

[[22]]
[1] "pdf/2008/08/04/The_Cartoon_Network_LP_LLLP_v._CSC_Holdings_Inc..pdf"

[[23]]
[1] "pdf/2008/06/25/Wheelabrator_Lisbon_Inc._v._State_of_CT_Dept._of_Pub._Util._Control.pdf"

[[24]]
[1] "pdf/2008/05/02/Ali_v._Mukasey_1.pdf"

[[25]]
[1] "pdf/2008/02/14/United_States_v._Verkhoglyad.pdf"

[[26]]
[1] "pdf/2007/11/08/United_States_v._Shellef_and_Rubenstein_1.pdf"

[[27]]
[1] "pdf/2007/10/12/Khulumani_v._Barclay_National_Bank_Ltd._Ntsebeza_v._Daimler_Chrysler_.pdf"

[[28]]
[1] "pdf/2007/04/23/ABN_Amro_Verzekeringen_BV_v._Geologistics_Ams._Inc._1.pdf"

[[29]]
[1] "pdf/2003/04/08/REENA_RAGGI_Circuit_Judge_concurring_in_the_judgment.pdf"

[[30]]
[1] "pdf/2010/04/23/United_States_of_America_v._Brown_et_al.pdf"

[[31]]
[1] "pdf/2010/04/22/Bautista-Ramirez_v._Holder.pdf"

[[32]]
[1] "pdf/2010/04/21/Vilkhu_v._City_of_New_York.pdf"

[[33]]
[1] "pdf/2010/04/21/Crenshaw_v._City_of_Mount_Vernon_et_al..pdf"

[[34]]
[1] "pdf/2010/04/20/Lesch_v._United_States_of_America.pdf"

[[35]]
[1] "pdf/2010/04/06/Islam_v._Holder.pdf"

[[36]]
[1] "pdf/2010/03/19/Kinneary_v._City_of_New_York.pdf"

[[37]]
[1] "pdf/2010/03/29/Anglo-Iberia_v._Lodderhose.pdf"

### Problem 2: "plain_text", "html", "html_lawbox", "html_columbia", "html_with_citations" each contain argument with different format

In [6]:
substr(opinions[[1]][c('plain_text','html','html_columbia','html_with_citations')],0,200)

[1] "          United States Court of Appeals\n                        For the First Circuit\n\n\nNos. 12-1573, 12-1653\n\n                    ELIZABETH MANNING ET AL.,\n\n                        Plaintiffs, Appel"  
[2] ""                                                                                                                                                                                                                  
[3] "NULL"                                                                                                                                                                                                              
[4] "<pre class=\"inline\">          United States Court of Appeals\n                        For the First Circuit\n\n\nNos. 12-1573, 12-1653\n\n                    ELIZABETH MANNING ET AL.,\n\n                     "

In [7]:
print_some_lines = function(x,n_lines) {
    print(paste('plain_text:',substr(x$plain_text,0,n_lines)))
    print(paste('html:',substr(x$html,0,n_lines)))
    print(paste('html_columbia:',substr(x$html_columbia,0,n_lines)))
    print(paste('html_with_citations:',substr(x$html_with_citations,0,n_lines)))
}

In [8]:
opinions[[4]]$absolute_url

[1] "/opinion/237855/doris-sylvia-grey-infant-and-howard-martin-grey-infant-children-of/"

In [9]:
print_some_lines(opinions[[4]],2000)

[1] "plain_text: "
[1] "html: <p class=\"case_cite\">227 F.2d 282</p>\n    <p class=\"parties\">Doris Sylvia GREY, infant, and Howard Martin Grey, infant,<br>children of Harry M. Goldberg, deceased, and<br>Sophie Goldberg, deceased, by Esther<br>Weiner, their guardian ad<br>litem, Plaintiffs-Appellants,<br>v.<br>AMERICAN AIRLINES, Inc., Defendant-Appellee.</p>\n    <p class=\"docket\">No. 73, Docket 23601.</p>\n    <p class=\"court\">United States Court of Appeals Second Circuit.</p>\n    <p class=\"date\">Argued Oct. 7, 1955.<br>Decided Nov. 7, 1955.</p>\n    <div class=\"prelims\">\n      <p class=\"indent\">Manes, Sturim, Donovan &amp; Laufer, New York City (Arthur M. Laufer and Samuel S. Sturim, New York City, on the brief), for plaintiffs-appellants.</p>\n      <p class=\"indent\">Haight, Gardner, Poor &amp; Havens, New York City (William J. Junkerman and James B. McQuillan, New York City, of counsel), for defendant-appellee.</p>\n      <p class=\"indent\">Before HAND, MEDINA and 

### Problem 3: Get Author Name

JSON files do have "author" and "author_str" keys, but they are empty...  
Metadata are available but professors won't share unless I am a co-author

In [10]:
opinions[[4]]$author
opinions[[4]]$author_str

NULL

[1] ""

https://www.courtlistener.com/opinion/237855/doris-sylvia-grey-infant-and-howard-martin-grey-infant-children-of/

Author is MEDINA not LUMBARD

In [11]:
substr(opinions[[4]]$html_with_citations,1025,1200)

[1] "<p class=\"indent\">Before HAND, MEDINA and LUMBARD, Circuit Judges.</p>\n      <p class=\"indent\">MEDINA, Circuit Judge.</p>\n    </div>\n    <div class=\"num\" id=\"p1\">\n      <span c"

In [12]:
str_match(opinions[[4]]$html_with_citations, "<p class=\"indent\"([ \t>]*)([a-zA-Z,]*)[ \t]Circuit[ \t]+Judge[.]")[,3]

[1] "MEDINA,"

### Problem 4: Dissents, Concurring, Per Curiam, Errata Sheets

Example Dissent: 

https://www.courtlistener.com/opinion/1036108/manning-v-boston-medical-center/

Example Errata Sheet:

https://www.courtlistener.com/opinion/1034770/in-reauerhahn-v/

Excluding these for now  

Total First Circuit Files: 34834  
Dissents: 275 (< 1%)  
Errata: 1527 (4%)  
Per Curiam: 3713 (10%)  

![image2](Progress_Reports/images/federal_dissents.png)

(From "Why (and when) judges dissents: A Theoretical Empiriccal Analysis" - Epstein, Landes, Posner (2011))

## Metadata

I created a metadata file (so far only includes first circuit):

Roughly 50,000 federal appellate court cases per year
Roughly 1400 first curcuit cases per year

In [13]:
load('data_inventory.ca1.RDATA')

In [14]:
head(df)

file_name,year,case_name,alt_case_name,circuit,local_path,absolute_url,type,author,joined_by,download_url,plain_text,judge,dissent,concurring,per_curiam,errata
1.json,2010,US_v._Davila-Gonzalez,united-states-v-davila-gonzalez,ca1,pdf/2010/02/10/US_v._Davila-Gonzalez.pdf,/opinion/1/united-states-v-davila-gonzalez/,010combined,NA,NA,http://www.ca1.uscourts.gov/pdf.opinions/08-2575P-01A.pdf,plain,"SELYA,",0,0,0,0
10.json,2010,US_v._Mitchell,united-states-v-mitchell,ca1,pdf/2010/02/22/US_v._Mitchell.pdf,/opinion/10/united-states-v-mitchell/,010combined,NA,NA,http://www.ca1.uscourts.gov/pdf.opinions/09-1260P-01A.pdf,plain,"TORRUELLA,",0,0,0,0
1000.json,2010,Adams_v._Adams,adams-v-adams,ca1,pdf/2010/03/31/Adams_v._Adams.pdf,/opinion/1000/adams-v-adams/,010combined,NA,NA,http://www.ca1.uscourts.gov/pdf.opinions/09-1443P-01A.pdf,plain,"STAHL,",0,0,0,0
1001.json,2010,Airframe_Systems_Inc._v._Raytheon_Company,airframe-systems-inc-v-raytheon-co,ca1,pdf/2010/03/31/Airframe_Systems_Inc._v._Raytheon_Company.pdf,/opinion/1001/airframe-systems-inc-v-raytheon-co/,010combined,NA,NA,http://www.ca1.uscourts.gov/pdf.opinions/09-1624P-01A.pdf,plain,"LYNCH,",0,0,0,0
1032057.json,2013,united_states_v._hogan,united-states-v-hogan,ca1,pdf/2013/07/05/united_states_v._hogan.pdf,/opinion/1032057/united-states-v-hogan/,010combined,NA,NA,http://media.ca1.uscourts.gov/pdf.opinions/12-1039P-01A.pdf,plain,"THOMPSON,",0,0,0,0
1032437.json,2013,mitchell_v._us_airways_inc.,mitchell-v-us-airways-inc,ca1,pdf/2013/07/09/mitchell_v._us_airways_inc..pdf,/opinion/1032437/mitchell-v-us-airways-inc/,010combined,NA,NA,http://media.ca1.uscourts.gov/pdf.opinions/12-1543P-01A.pdf,plain,"SELYA,",0,0,0,0


In [15]:
nrow(df)

[1] 34834

In [16]:
sum(is.na(df$year))

[1] 21190

In [17]:
sum(is.na(df$local_path))

[1] 21190

In [18]:
sum(is.na(df$plain_text))

[1] 1102

## Text Model

![doc2vec](Progress_Reports/images/doc2vec.png)

## Analysis

Average document vectors for each judge then plot.
Only keep judge if more than 100 opinions are available. (25 judges)

In [19]:
# Load Data
judge_meta = read.csv(file='ca1_Judge_Metadata.csv')
head(judge_meta,30)

Judge,Judge.full,Born,Active,Appointed.By,X
putnam,William LeBaron Putnam,1835,1892–1917,B. Harrison,0
colt,LeBaron B. Colt,1846,1891–1913[Note 1],Arthur,0
dodge,Frederic Dodge,1847,1912–1918,Taft,0
lowell,Francis Cabot Lowell,1855,1905–1911,T. Roosevelt,0
schofield,William Schofield,1857,1911–1912,Taft,0
johnson,Charles Fletcher Johnson,1859,1917–1929,Wilson,1
anderson,George Weston Anderson,1861,1918–1931,Wilson,0
bingham,George Hutchins Bingham,1864,1913–1939,Wilson,1
morton,James Madison Morton Jr.,1869,1932–1939,Hoover,1
wilson,Scott Wilson,1870,1929–1940,Hoover,1


![pca](Progress_Reports/images/pca.png)

Next: de-mean by decade, circuit court, THEN average by judge.

If possible, de-mean by topic as well.

## Code Schema

![schema](Progress_Reports/images/schema.png)